In [0]:
df = (
    spark.read
         .option("header", "true")
         .option("inferSchema", "true")
         .csv("dbfs:/Volumes/zillow/zillow_medallion/raw/County_time_series.csv")
)

display(df)


In [0]:
entry_count = df.count()
print("Number of entries in df:", entry_count)

In [0]:
RAW = "/Volumes/zillow/zillow_medallion/raw"
OUT_FILE = "/Volumes/zillow/zillow_medallion/raw-converted_format/json/County_time_series.json"
OUT_DIR = "/Volumes/zillow/zillow_medallion/raw-converted_format/json"
TMP_DIR = "/Volumes/zillow/zillow_medallion/raw-converted_format/json/_tmp_county_json"

src = f"{RAW}/County_time_series.csv"

# Clean output
dbutils.fs.mkdirs("dbfs:" + OUT_DIR)
try: dbutils.fs.rm("dbfs:" + OUT_FILE, True)
except: pass
dbutils.fs.rm("dbfs:" + TMP_DIR, True)

df = (spark.read.option("header", "true").option("inferSchema", "true").csv(src))

# Write as single JSON part file (JSON lines)
df.coalesce(1).write.mode("overwrite").json(TMP_DIR)

files = dbutils.fs.ls("dbfs:" + TMP_DIR)
part_file = [f.path for f in files if f.name.startswith("part-") and f.name.endswith(".json")][0]

dbutils.fs.mv(part_file, "dbfs:" + OUT_FILE, True)
dbutils.fs.rm("dbfs:" + TMP_DIR, True)

print("Wrote JSON:", OUT_FILE)
display(dbutils.fs.ls("dbfs:" + OUT_DIR))


In [0]:
df = spark.read.json("/Volumes/zillow/zillow_medallion/raw-converted_format/json/County_time_series.json")
display(df)

In [0]:
entry_count = df.count()
print("Number of entries in df:", entry_count)